In [13]:
%matplotlib inline
from urllib import request, parse
from bs4 import BeautifulSoup
from pymongo import MongoClient
import datetime
import ssl
import re
import time
import json
import random
import pprint

In [18]:
ssl._create_default_https_context = ssl._create_unverified_context  # 全局取消网页安全验证

In [2]:
client = MongoClient('localhost', 27017)
db = client.moniter_crowdfunding
project = db.projects
f_project = db.failure_projects

In [3]:
p_founder = db.founders

## 单个项目

In [165]:
item = project.find_one({'状态': '众筹成功'}, projection={'公司名称':1, '公司地址':1, '公司工作时间':1, 
                                                        '公司电话':1, '发起人名称':1, '发起人链接':1})

In [166]:
prj_info = project.find_one({'_id':item['_id']}, 
                            projection={'状态':1, '所属类别':1, '项目名称':1, '众筹期限':1, '目标金额':1, '_id':0})

- Head信息

In [167]:
User_Agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:60.0) Gecko/20100101 Firefox/60.0'
Host = 'z.jd.com'
craw_time = datetime.datetime.now()

- 获取网页内容

In [168]:
def crawData(href, _id):
    post_data = parse.urlencode({"flag":"2", "id":_id})
    req = request.Request(href)
    req.add_header('User-Agent', User_Agent)
    req.add_header('Referer', f'http://z.jd.com/funderCenter.action?{post_data}')
    req.add_header('Host', Host)
    with request.urlopen(req, data=post_data.encode('utf-8')) as f:
        raw_html = f.read().decode()
        
    return raw_html

- 解析获取数据

In [199]:
def soupData(raw):
    h_soup = BeautifulSoup(raw, 'html.parser')
    cat_div = h_soup.find('div', {'class': 'tab_cont db'})
    p_d = re.compile('(\d+)')
    
    c_dict = {}
    if cat_div:
        for x in cat_div.findAll('li'):
            prj_href = str(x).split("'")[1]
            prj_id = p_d.findall(prj_href)[0]
            c_dict[prj_id] = {'prj_name': x.p.string, 'prj_desc': x.a.string, 'prj_href': prj_href}
        
    return c_dict

In [200]:
item = {'_id': '83904'}
hrefs = {'支持项目': 'http://z.jd.com/f/my_support.action?',
         '关注项目': 'http://z.jd.com/f/my_focus.action?', 
         '发起项目': 'http://z.jd.com/f/my_project.action?'}

for key in hrefs:
    print(key)
    raw = crawData(hrefs[key], item['_id'])
    c_dict = soupData(raw)
    item[key] = c_dict

支持项目
关注项目
发起项目


In [201]:
item

{'_id': '83904',
 '支持项目': {},
 '关注项目': {'83904': {'prj_name': '小牙兽-防龋齿智能幼教牙刷',
   'prj_desc': '一支乳牙期儿童智能牙刷，容纳了怎样的智能？巴氏刷牙法口腔游戏教程+儿童的专属口腔银行+您身边的牙医绿色通道，让儿童自动养成快乐自律的口腔护理习惯',
   'prj_href': '/project/details/83904.html'}},
 '发起项目': {'83904': {'prj_name': '小牙兽-防龋齿智能幼教牙刷',
   'prj_desc': '一支乳牙期儿童智能牙刷，容纳了怎样的智能？巴氏刷牙法口腔游戏教程+儿童的专属口腔银行+您身边的牙医绿色通道，让儿童自动养成快乐自律的口腔护理习惯',
   'prj_href': '/project/details/83904.html'}}}

## 查看所有项目发起人的主页

In [218]:
from bson import ObjectId

- 初始化: 读取其它数据集中已有的id 

In [328]:
def checkOtherCols():
        suc_ids = list(set(x['_id'] for x in project.find({}, projection={'_id':1})))
        for p_id in suc_ids[::-1]:
            if isinstance(p_id, ObjectId):
                print(f"success {p_id}")
                t_id = project.find_one({"_id":p_id}, projection={'项目编号':1})
                suc_ids.remove(p_id)
                suc_ids.append(t_id['项目编号'])
        
        fail_ids = list(set(x['详细信息']['_id'] for x in f_project.find({}, projection={'详细信息._id':1})))
        for p_id in fail_ids[::-1]:  # 将_id为ObjectID类型的替换为项目编号
            if isinstance(p_id, ObjectId):
                print(f"failure {p_id}")
                t_id = f_project.find_one({"详细信息._id":p_id}, projection={"详细信息.项目编号":1})
                fail_ids.remove(p_id)
                fail_ids.append(t_id["详细信息"]["项目编号"])
                
        return set(suc_ids) | set(fail_ids)

In [329]:
ids = checkOtherCols()

success 5af292a738ea07432da62d98
success 5af292a738ea07432da62d93
success 5aeb00d458abe024f8d01956
success 5af292a738ea07432da62d95
success 5af292a738ea07432da62d9a
success 5af292a638ea07432da62d91
success 5af292a738ea07432da62d99
success 5af292a638ea07432da62d90
success 5af292a738ea07432da62d94
success 5af292a738ea07432da62d97
success 5af292a738ea07432da62d92


In [301]:
suc_ids = [x['_id'] for x in project.find({}, projection={'_id':1})]
fail_ids = [x['详细信息']['_id'] for x in f_project.find({}, projection={'详细信息._id':1})]
ids = suc_ids + fail_ids

for p_id in ids[::-1]:  # 将_id为ObjectID类型的替换为项目编号
    if isinstance(p_id, ObjectId):
        print(p_id)
        t_id = project.find_one({"_id":p_id}, projection={'项目编号':1})
        ids.remove(p_id)
        ids.append(t_id['项目编号'])

5af292a738ea07432da62d9a
5af292a738ea07432da62d99
5af292a738ea07432da62d98
5af292a738ea07432da62d97
5aeb00d458abe024f8d01956
5af292a738ea07432da62d95
5af292a738ea07432da62d94
5af292a738ea07432da62d93
5af292a738ea07432da62d92
5af292a638ea07432da62d91
5af292a638ea07432da62d90


In [302]:
ids = set(ids)
hrefs = {'支持项目': 'http://z.jd.com/f/my_support.action?',
        '关注项目': 'http://z.jd.com/f/my_focus.action?', 
        '发起项目': 'http://z.jd.com/f/my_project.action?'}

- 插入1个文档

In [308]:
def insert_data(p_id):
    item = {'_id': p_id}
    try:
        for key in hrefs:
            raw = crawData(hrefs[key], item['_id'])
            c_dict = soupData(raw)
            item[key] = c_dict
        p_founder.insert_one(item)
        return 1
    except Exception as e:
        print(f"{p_id}插入失败!错误提示:{e}")
        return 0

- 插入多个文档

In [316]:
def insert_dataset(p_ids):
    inserted, notInserted = [], []
    for p_id in p_ids:
        if insert_data(p_id):
            inserted.append(p_id)
        else:
            notInserted.append(p_id)
            
    return inserted, notInserted

- 计算新生成的id

In [305]:
def get_newIDs(p_founder=p_founder):
    now_ids = []
    new_res = []
    for x in p_founder.find({}):
        temp = list(x['支持项目'].keys()) + list(x['关注项目'].keys()) + list(x['发起项目'].keys())
        now_ids.append(x['_id'])
        new_res.extend(temp)
        
    return set(new_res) - set(now_ids)

- 循环爬取数据

In [312]:
j = 1
while True:
    time.sleep(10)
    print(f"第{j}轮")
    newIDs = get_newIDs()
    if newIDs:
        inserted, notinserted = insert_dataset(newIDs)
        j += 1
    else:
        break

第1轮
计划插入了461个文档
实际插入了461个文档, 用时 13.67秒。
第2轮
计划插入了338个文档
实际插入了338个文档, 用时 13.12秒。
第3轮
计划插入了236个文档
实际插入了236个文档, 用时 7.33秒。
第4轮
计划插入了160个文档
实际插入了160个文档, 用时 5.10秒。
第5轮
计划插入了139个文档
实际插入了139个文档, 用时 4.90秒。
第6轮
计划插入了61个文档
实际插入了61个文档, 用时 1.69秒。
第7轮
计划插入了28个文档
实际插入了28个文档, 用时 0.85秒。
第8轮
计划插入了7个文档
实际插入了7个文档, 用时 0.17秒。
第9轮


In [315]:
p_founder.count_documents({})

6413

### 转化为网络

In [252]:
import networkx as nx

In [274]:
dg = nx.DiGraph()

In [276]:
try:
    x = 1/0
except Exception as e:
    print(e)

division by zero


In [292]:
a = {1, 2}
a.update([3,4])
a

{1, 2, 3, 4}

In [322]:
fail_ids = set(x['项目编号'] for x in f_project.find({}, projection={"项目编号":1, '详细信息._id':1}))

In [323]:
fail_ids

{'100006',
 '100009',
 '100016',
 '100029',
 '100066',
 '100068',
 '100069',
 '100095',
 '100098',
 '100113',
 '100136',
 '100161',
 '100183',
 '100192',
 '100212',
 '100222',
 '100237',
 '100244',
 '100254',
 '100256',
 '100280',
 '100282',
 '100285',
 '100300',
 '100311',
 '100312',
 '100321',
 '100338',
 '100372',
 '100382',
 '100427',
 '100444',
 '100454',
 '100464',
 '100465',
 '100491',
 '100494',
 '100495',
 '100535',
 '100559',
 '100583',
 '100586',
 '100597',
 '100598',
 '100610',
 '100622',
 '100642',
 '100648',
 '100663',
 '100679',
 '100695',
 '100714',
 '100723',
 '100738',
 '100774',
 '100808',
 '100832',
 '100843',
 '100860',
 '100903',
 '100911',
 '100914',
 '100917',
 '100932',
 '100956',
 '100959',
 '100966',
 '100973',
 '100977',
 '101003',
 '101034',
 '101041',
 '101045',
 '101048',
 '101062',
 '101068',
 '101072',
 '101080',
 '101106',
 '101122',
 '101173',
 '101217',
 '101219',
 '101233',
 '101247',
 '101248',
 '101252',
 '101256',
 '101271',
 '101279',
 '101295',